<a href="https://colab.research.google.com/github/mengcius/pytorch-learn/blob/master/12_%E5%8D%B7%E7%A7%AF%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 12_卷积神经网络

### 卷积网络层

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

类风格实现（nn.Xxx）：要先实例化再调用

In [0]:
x=torch.rand(1,1,28,28) #核个数，输入通道数，核大小
layer=nn.Conv2d(1,3,kernel_size=3,stride=1,padding=0) #卷积层，类风格实现：输入通道数，核数量

out=layer.forward(x) #卷积前向运算（不建议用）
out.shape

torch.Size([1, 3, 26, 26])

In [0]:
out=layer(x) #卷积前向运算，用封装好的方法，直接是实例+()
out.shape

torch.Size([1, 3, 26, 26])

In [0]:
layer.weight #查看网络层里的参数，[3,1,3,3]核数量 输入通道 核大小，w b会自动跟新

Parameter containing:
tensor([[[[-0.3300,  0.0198, -0.1243],
          [ 0.0597, -0.0743,  0.2338],
          [-0.2747,  0.2101,  0.2337]]],


        [[[-0.2405,  0.1064,  0.0402],
          [ 0.3295,  0.1337,  0.0645],
          [ 0.0190, -0.0427,  0.1558]]],


        [[[-0.3241,  0.0522, -0.0070],
          [ 0.0812, -0.2457, -0.1421],
          [ 0.1151,  0.0057, -0.1088]]]], requires_grad=True)

函数风格实现（F.xxx）：自己管理这里tensor、运算过程，仅仅用了它GUP加速的过程，不用实例化。

pytorch默认大写是类，小写是接口

In [0]:
w=torch.rand(16,3,5,5) #核个数，输入通道数，核大小
b=torch.rand(16)
x=torch.randn(1,3,28,28) #核个数，输入通道数，核大小

out=F.conv2d(x,w,b,stride=1,padding=1) #CNN的一层，底层函数风格实现
out.shape

torch.Size([1, 16, 26, 26])

### 上下采样

池化

In [0]:
x=torch.rand(1,16,14,14)

In [0]:
layer=nn.MaxPool2d(2,stride=2) #最大值池化，类风格实现：窗口大小，步长
out=layer(x)
out.shape #减半了

torch.Size([1, 16, 7, 7])

In [0]:
layer=F.avg_pool2d(x,2,stride=2) #最大值池化，函数风格实现
out.shape

torch.Size([1, 16, 7, 7])

上采样

In [0]:
out=F.interpolate(x,scale_factor=3,mode='nearest') #插值法上采样：tensor，放大倍数，插值方法（近邻、双性、三性）
out.shape #

torch.Size([1, 16, 42, 42])

### ReLU函数

In [0]:
x.shape

torch.Size([1, 16, 14, 14])

In [0]:
layer=nn.ReLU(inplace=True) #ReLU函数，维度不变，inplace=True就是让结果替换原来的x的地址，节省内存
out=layer(x)
out.shape

torch.Size([1, 16, 14, 14])

In [0]:
layer=F.relu(x)
out.shape

torch.Size([1, 16, 14, 14])

### Batch Normalization

一维

In [0]:
x=torch.rand(100,16,784) #均匀分布的均值是0.5，方差为1
layer=nn.BatchNorm1d(16) #Batch Normal;用1d是因为1维的feature 28*28，16就是通道数
out=layer(x) #统计出当前x的u和o^2
out.shape

torch.Size([100, 16, 784])

In [0]:
layer.running_mean #更新全局的running_u，当只有一次运算时就是u，都在0.5附近

tensor([0.0500, 0.0499, 0.0498, 0.0500, 0.0500, 0.0499, 0.0497, 0.0499, 0.0498,
        0.0500, 0.0500, 0.0498, 0.0499, 0.0498, 0.0499, 0.0500])

In [0]:
layer.running_var #更新全局的running_o^2，方差为1左右

tensor([0.9083, 0.9084, 0.9083, 0.9083, 0.9083, 0.9083, 0.9084, 0.9083, 0.9083,
        0.9083, 0.9083, 0.9083, 0.9083, 0.9083, 0.9084, 0.9083])

二维

In [0]:
x=torch.rand(1,16,7,7)
x.shape

torch.Size([1, 16, 7, 7])

In [0]:
layer=nn.BatchNorm2d(16)
out=layer(x)
out.shape #大小不变

torch.Size([1, 16, 7, 7])

In [0]:
layer.weight #相当于r，需要梯度

Parameter containing:
tensor([0.3847, 0.9785, 0.7068, 0.7617, 0.5684, 0.7968, 0.0674, 0.8322, 0.9962,
        0.2803, 0.3346, 0.1811, 0.4323, 0.6628, 0.6418, 0.6469],
       requires_grad=True)

In [0]:
layer.bias #相当于B

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       requires_grad=True)

In [0]:
vars(layer) #Class variables

{'_backend': <torch.nn.backends.thnn.THNNFunctionBackend at 0x7f5edd095668>,
 '_backward_hooks': OrderedDict(),
 '_buffers': OrderedDict([('running_mean',
               tensor([0.0450, 0.0558, 0.0435, 0.0471, 0.0516, 0.0510, 0.0461, 0.0421, 0.0493,
                       0.0516, 0.0499, 0.0498, 0.0552, 0.0517, 0.0464, 0.0504])),
              ('running_var',
               tensor([0.9090, 0.9087, 0.9091, 0.9075, 0.9097, 0.9088, 0.9078, 0.9065, 0.9085,
                       0.9097, 0.9072, 0.9109, 0.9090, 0.9070, 0.9080, 0.9054])),
              ('num_batches_tracked', tensor(1))]),
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_modules': OrderedDict(),
 '_parameters': OrderedDict([('weight', Parameter containing:
               tensor([0.3847, 0.9785, 0.7068, 0.7617, 0.5684, 0.7968, 0.0674, 0.8322, 0.9962,
                       0.2803, 0.3346, 0.1811, 0.4323, 0.6628, 0.6418, 0.6469],
                      requ

测试里的Batch Norm

In [0]:
layer.eval()
nn.BatchNorm1d(16,eps=1e-05,momentum=0.1,affine=True,track_running_stats=True)
out=layer(x)
out.shape

torch.Size([1, 16, 7, 7])

### ResNet的残差单元

In [0]:
import  torch
from    torch import  nn
from    torch.nn import functional as F

# from    torchvision.models import resnet18

# 残差单元（适用于浅层网络，中间有两层卷积层）
class ResBlk(nn.Module):
    """
    resnet block
    """

    def __init__(self, ch_in, ch_out):
        """
        :param ch_in:
        :param ch_out:
        """
        super(ResBlk, self).__init__()

        self.conv1 = nn.Conv2d(ch_in, ch_out, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(ch_out)
        self.conv2 = nn.Conv2d(ch_out, ch_out, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(ch_out)

        self.extra = nn.Sequential()
        if ch_out != ch_in: # 确保x跳连的输出是ch_out，以便与卷积层像素相加
            # [b, ch_in, h, w] => [b, ch_out, h, w]
            self.extra = nn.Sequential(
                nn.Conv2d(ch_in, ch_out, kernel_size=1, stride=1),
                nn.BatchNorm2d(ch_out)
            )

    def forward(self, x):
        """
        :param x: [b, ch, h, w]
        :return:
        """
        out = F.relu(self.bn1(self.conv1(x))) # 这里加relu
        out = self.bn2(self.conv2(out))
        # short cut，extra module: [b, ch_in, h, w] => [b, ch_out, h, w]
        out = self.extra(x) + out # 元素相加（element-wise add）

        return out

### ResNet网络
ResNet18跑CIFAR10，test acc几乎没变，train loss在260 epoch就为0了

259 loss: 0.0
259 acc: 0.5816
260 loss: 0.0
260 acc: 0.5812
261 loss: 0.0
261 acc: 0.579

699 loss: 0.0
699 acc: 0.5886
700 loss: 0.0
700 acc: 0.5784
701 loss: 0.0
701 acc: 0.582
702 loss: 2.384185791015625e-06
702 acc: 0.5793

In [0]:
import  torch
from    torch import  nn
from    torch.nn import functional as F
from    torch.utils.data import DataLoader
from    torchvision import datasets
from    torchvision import transforms
from    torch import nn, optim

# from    torchvision.models import resnet18

# 残差单元（适用于浅层网络，中间有两层卷积层）
class ResBlk(nn.Module):
    """
    resnet block
    """

    def __init__(self, ch_in, ch_out):
        """
        :param ch_in:
        :param ch_out:
        """
        super(ResBlk, self).__init__()

        self.conv1 = nn.Conv2d(ch_in, ch_out, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(ch_out)
        self.conv2 = nn.Conv2d(ch_out, ch_out, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(ch_out)

        self.extra = nn.Sequential()
        if ch_out != ch_in: # 确保x跳连的输出是ch_out，以便与卷积层像素相加
            # [b, ch_in, h, w] => [b, ch_out, h, w]
            self.extra = nn.Sequential(
                nn.Conv2d(ch_in, ch_out, kernel_size=1, stride=1),
                nn.BatchNorm2d(ch_out)
            )

    def forward(self, x):
        """
        :param x: [b, ch, h, w]
        :return:
        """
        out = F.relu(self.bn1(self.conv1(x))) # 这里加relu
        out = self.bn2(self.conv2(out))
        # short cut，extra module: [b, ch_in, h, w] => [b, ch_out, h, w]
        out = self.extra(x) + out # 元素相加（element-wise add）

        return out


# ResNet18网络结构
class ResNet18(nn.Module):

    def __init__(self):
        super(ResNet18, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16)
        )
        # followed 4 blocks
        # [b, 64, h, w] => [b, 128, h ,w]
        self.blk1 = ResBlk(16, 16)
        # [b, 128, h, w] => [b, 256, h, w]
        self.blk2 = ResBlk(16, 32)
        # # [b, 256, h, w] => [b, 512, h, w]
        # self.blk3 = ResBlk(128, 256)
        # # [b, 512, h, w] => [b, 1024, h, w]
        # self.blk4 = ResBlk(256, 512)

        self.outlayer = nn.Linear(32*32*32, 10)

    def forward(self, x):
        """
        :param x:
        :return:
        """
        x = F.relu(self.conv1(x))

        # [b, 64, h, w] => [b, 1024, h, w]
        x = self.blk1(x)
        x = self.blk2(x)
        # x = self.blk3(x)
        # x = self.blk4(x)

        # print(x.shape)
        x = x.view(x.size(0), -1)
        x = self.outlayer(x)

        return x


def main():
    batchsz = 32

    cifar_train = datasets.CIFAR10('cifar', True, transform=transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor()
    ]), download=True)
    cifar_train = DataLoader(cifar_train, batch_size=batchsz, shuffle=True)

    cifar_test = datasets.CIFAR10('cifar', False, transform=transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor()
    ]), download=True)
    cifar_test = DataLoader(cifar_test, batch_size=batchsz, shuffle=True)


    x, label = iter(cifar_train).next()
    print('x:', x.shape, 'label:', label.shape)

    device = torch.device('cuda')
    # model = Lenet5().to(device)
    model = ResNet18().to(device)

    criteon = nn.CrossEntropyLoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    print(model)

    for epoch in range(1000):

        model.train()
        for batchidx, (x, label) in enumerate(cifar_train):
            # [b, 3, 32, 32]
            # [b]
            x, label = x.to(device), label.to(device)

            logits = model(x)
            # logits: [b, 10]
            # label:  [b]
            # loss: tensor scalar
            loss = criteon(logits, label)

            # backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        #
        print(epoch, 'loss:', loss.item())


        model.eval()
        with torch.no_grad():
            # test
            total_correct = 0
            total_num = 0
            for x, label in cifar_test:
                # [b, 3, 32, 32]
                # [b]
                x, label = x.to(device), label.to(device)

                # [b, 10]
                logits = model(x)
                # [b]
                pred = logits.argmax(dim=1)
                # [b] vs [b] => scalar tensor
                correct = torch.eq(pred, label).float().sum().item()
                total_correct += correct
                total_num += x.size(0)
                # print(correct)

            acc = total_correct / total_num
            print(epoch, 'acc:', acc)


if __name__ == '__main__':
    main()

0it [00:00, ?it/s]

170500096it [00:06, 27783743.02it/s]                               


Files already downloaded and verified
x: torch.Size([32, 3, 32, 32]) label: torch.Size([32])
ResNet18(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (blk1): ResBlk(
    (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (extra): Sequential()
  )
  (blk2): ResBlk(
    (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, 

### nn.Module

In [1]:
import  torch
from    torch import nn
from    torch import optim

# 自定义线性层
class MyLinear(nn.Module):

    def __init__(self, inp, outp):
        super(MyLinear, self).__init__()

        # requires_grad = True
        self.w = nn.Parameter(torch.randn(outp, inp))
        self.b = nn.Parameter(torch.randn(outp))

    def forward(self, x):
        x = x @ self.w.t() + self.b
        return x

# 展平
class Flatten(nn.Module):

    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, input):
        return input.view(input.size(0), -1)

class TestNet(nn.Module):

    def __init__(self):
        super(TestNet, self).__init__()

        self.net = nn.Sequential(nn.Conv2d(1, 16, stride=1, padding=1),
                                 nn.MaxPool2d(2, 2),
                                 Flatten(), # 展平
                                 nn.Linear(1*14*14, 10))

    def forward(self, x):
        return self.net(x)


class BasicNet(nn.Module):

    def __init__(self):
        super(BasicNet, self).__init__()

        self.net = nn.Linear(4, 3)

    def forward(self, x):
        return self.net(x)

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.net = nn.Sequential(BasicNet(),
                                 nn.ReLU(),
                                 nn.Linear(3, 2))

    def forward(self, x):
        return self.net(x)


def main():
    device = torch.device('cuda')
    net = Net()
    net.to(device)

    net.train()

    net.eval()

    # net.load_state_dict(torch.load('ckpt.mdl'))
    #
    #
    # torch.save(net.state_dict(), 'ckpt.mdl')

    for name, t in net.named_parameters():
        print('parameters:', name, t.shape)

    for name, m in net.named_children():
        print('children:', name, m)

    for name, m in net.named_modules():
        print('modules:', name, m)


if __name__ == '__main__':
    main()

parameters: net.0.net.weight torch.Size([3, 4])
parameters: net.0.net.bias torch.Size([3])
parameters: net.2.weight torch.Size([2, 3])
parameters: net.2.bias torch.Size([2])
children: net Sequential(
  (0): BasicNet(
    (net): Linear(in_features=4, out_features=3, bias=True)
  )
  (1): ReLU()
  (2): Linear(in_features=3, out_features=2, bias=True)
)
modules:  Net(
  (net): Sequential(
    (0): BasicNet(
      (net): Linear(in_features=4, out_features=3, bias=True)
    )
    (1): ReLU()
    (2): Linear(in_features=3, out_features=2, bias=True)
  )
)
modules: net Sequential(
  (0): BasicNet(
    (net): Linear(in_features=4, out_features=3, bias=True)
  )
  (1): ReLU()
  (2): Linear(in_features=3, out_features=2, bias=True)
)
modules: net.0 BasicNet(
  (net): Linear(in_features=4, out_features=3, bias=True)
)
modules: net.0.net Linear(in_features=4, out_features=3, bias=True)
modules: net.1 ReLU()
modules: net.2 Linear(in_features=3, out_features=2, bias=True)


### 数据增强
翻转、旋转、缩放、剪裁

效果并不好

In [3]:
! npm install -g localtunnel
# 8097可修改为自己要用的端口号
get_ipython().system_raw('python3 -m pip install visdom')
get_ipython().system_raw('python3 -m visdom.server -port 8097 >> visdomlog.txt 2>&1 &')
get_ipython().system_raw('lt --port 8097 >> url.txt 2>&1 &')
! cat url.txt
# import visdom
# vis = visdom.Visdom(port='8097')
# vis.text('testing')
# ! cat visdomlog.txt

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.0
added 35 packages from 21 contributors in 2.024s
/bin/bash: lt: command not found
your url is: https://big-eagle-26.localtunnel.me


In [7]:
import  torch
import  torch.nn as nn
import  torch.nn.functional as F
import  torch.optim as optim
from    torchvision import datasets, transforms

from visdom import Visdom

batch_size=200
learning_rate=0.01
epochs=10

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('mnist_data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.RandomHorizontalFlip(), # 水平翻转，Random是随机做
                       transforms.RandomVerticalFlip(), # 数值翻转
                       transforms.RandomRotation(15), # 随机旋转-15~15度之间任意角度
                       transforms.RandomRotation([90, 270]), # 随机旋转这几个选项中的一个角度（只能有两个值）
                    #    transforms.Resize([32, 32]), # 缩放
                    #    transforms.RandomCrop([28, 28]), # 裁剪
                       transforms.ToTensor(), # numpy格式转为Tensor
                       # transforms.Normalize((0.1307,), (0.3081,)) # 归一化
                   ])),
    batch_size=batch_size, shuffle=True) # 一次加载batch_size张图片，随机打散

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('mnist_data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        # transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=True)


# 全连接网络
class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(784, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 10),
            nn.LeakyReLU(inplace=True),
        )

    def forward(self, x):
        x = self.model(x)

        return x

device = torch.device('cuda:0')
net = MLP().to(device)
optimizer = optim.SGD(net.parameters(), lr=learning_rate)
criteon = nn.CrossEntropyLoss().to(device)

viz = Visdom(port='8097') # Visdom实例化，加上端口号
# 先创建一条曲线，后面再把最新数据添加到上面去。参数：初始点坐标y x，win是小窗口ID，窗口名称
viz.line([0.], [0.], win='train_loss', opts=dict(title='train loss'))
# 一个窗口同时显示两条曲线，参数：初始点坐标y1 y2 x，ID，两个窗口名称，两个图例说明
viz.line([[0.0, 0.0]], [0.], win='test', opts=dict(title='test loss&acc.',legend=['loss', 'acc.'])) 
global_step = 0 # x轴

for epoch in range(epochs):

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28*28)
        data, target = data.to(device), target.cuda()

        logits = net(data)
        loss = criteon(logits, target)

        optimizer.zero_grad()
        loss.backward()
        # print(w1.grad.norm(), w2.grad.norm())
        optimizer.step()

        #添加新数据到曲线上：loss是numpy类型，x轴时间戳，ID，更新方式为往后添加，否则就会覆盖掉
        global_step += 1 #x加1，这里有3000个,3000*200=60000*10
        viz.line([loss.item()], [global_step], win='train_loss', update='append') 

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))

    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.view(-1, 28 * 28)
        data, target = data.to(device), target.cuda()
        logits = net(data)
        test_loss += criteon(logits, target).item()

        pred = logits.argmax(dim=1)
        correct += pred.eq(target).float().sum().item()

    #添加新数据到曲线上，y1 y2 x，最好y1和y2的范围接近
    viz.line([[test_loss, correct/len(test_loader.dataset)]], [global_step], win='test', update='append')

    #图片可视化，这里显示test_loader最近一批的200张
    viz.images(data.view(-1, 1, 28, 28), win='x') #变形为图片，图像可以直接传tensor类型不用转numpy
    viz.text(str(pred.detach().cpu().numpy()), win='pred', opts=dict(title='pred')) #预测的标签，要转numpy

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Setting up a new session...


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.304445
Train Epoch: 0 [20000/60000 (33%)]	Loss: 2.292063
Train Epoch: 0 [40000/60000 (67%)]	Loss: 2.285799

Test set: Average loss: 0.0113, Accuracy: 2999.0/10000 (30%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.275401
Train Epoch: 1 [20000/60000 (33%)]	Loss: 2.239706
Train Epoch: 1 [40000/60000 (67%)]	Loss: 2.201200

Test set: Average loss: 0.0108, Accuracy: 3252.0/10000 (33%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.152547
Train Epoch: 2 [20000/60000 (33%)]	Loss: 2.089677
Train Epoch: 2 [40000/60000 (67%)]	Loss: 2.045973

Test set: Average loss: 0.0096, Accuracy: 4201.0/10000 (42%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 1.900767
Train Epoch: 3 [20000/60000 (33%)]	Loss: 1.835191
Train Epoch: 3 [40000/60000 (67%)]	Loss: 1.728176

Test set: Average loss: 0.0080, Accuracy: 4638.0/10000 (46%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 1.646666
Train Epoch: 4 [20000/60000 (33%)]	Loss: 1.635103
Train Epoch: 4 [40000/60000 (67%)]	Loss: 1.531050

Test set: Average loss